### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; updated 10.31.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import uniform
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run ../common_functions.ipynb

In [4]:
run ../priors_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [9]:
#sim_data(15,[.2,.3,2,.5, .8, .2])

### Pull in best-fit parameters from participants

In [10]:
dt1 = pd.read_csv('../../../output/prior_model_params_younger.csv')
dt2 = pd.read_csv('../../../output/prior_model_params_older.csv')
dt = pd.concat([dt1, dt2], ignore_index = True)
print(dt)

          id  alpha_gain  alpha_loss      beta  iProb_trust  iProb_neut  \
0   sub-1009    0.886495    0.012511  0.126113     0.987471    0.987153   
1   sub-1021    0.139032    0.337882  0.532192     0.996336    0.522594   
2   sub-1035    0.278623    0.423078  0.379328     0.692258    0.983758   
3   sub-1034    0.298891    0.663975  0.434164     0.986609    0.766490   
4   sub-1020    0.293184    0.748133  0.485972     1.000000    0.062512   
..       ...         ...         ...       ...          ...         ...   
58  sub-2009    0.003951    0.398593  0.090728     0.037509    0.018708   
59  sub-2037    0.237740    0.858579  0.290348     0.133304    0.106669   
60  sub-2023    0.166507    0.350909  0.704742     0.630379    0.559936   
61  sub-2022    0.002954    0.662100  0.038958     0.027248    0.016404   
62  sub-2036    0.712716    0.010051  0.144787     1.000000    0.999152   

    iProb_untrust       -LLH  
0    1.274585e-05  57.315963  
1    2.059150e-02  37.246681  
2    1

In [13]:
a_gains = dt['alpha_gain']
a_losses = dt['alpha_loss']
betas = dt['beta']
iprobA = dt['iProb_trust']
iprobB = dt['iProb_neut']
iprobC = dt['iProb_untrust']
parent_sub = dt['id']
print(a_gains)

0     0.886495
1     0.139032
2     0.278623
3     0.298891
4     0.293184
        ...   
58    0.003951
59    0.237740
60    0.166507
61    0.002954
62    0.712716
Name: alpha_gain, Length: 63, dtype: float64


### Simulate data

In [16]:
data = pd.DataFrame(columns= ['Trial','Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 
                              'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
for p in range(len(betas)):
    for q in range(1,11):
        dt = sim_data(15, [a_gains[p], a_losses[p], betas[p], iprobA[p], iprobB[p], iprobC[p]])
        dt['Subject'] = parent_sub[p] + '_' + str(q)
        dt['Alpha_gain'] = a_gains[p] 
        dt['Alpha_loss'] = a_losses[p]
        dt['Beta'] = betas[p]
        dt['iProbA'] = iprobA[p]
        dt['iProbB'] =  iprobB[p]
        dt['iProbC'] = iprobC[p]
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])

data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_2alpha_with_priors_model_data.csv', index = False)

In [15]:
print(data)

   Trial Stim_Sequence Choice Trustee_Response      Subject  Alpha_gain  \
0      1             1      1                0   sub-1009_1    0.886495   
1      2             0      1                1   sub-1009_1    0.886495   
2      3             2      3                0   sub-1009_1    0.886495   
3      4             0      1                1   sub-1009_1    0.886495   
4      5             1      3                0   sub-1009_1    0.886495   
..   ...           ...    ...              ...          ...         ...   
40    41             2      4                0  sub-2036_10    0.712716   
41    42             2      2                0  sub-2036_10    0.712716   
42    43             2      4                0  sub-2036_10    0.712716   
43    44             0      2                1  sub-2036_10    0.712716   
44    45             1      3                1  sub-2036_10    0.712716   

    Alpha_loss      Beta    iProbA    iProbB    iProbC  
0     0.012511  0.126113  0.987471  0.9871